In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import random
from scipy import stats
import admix_prs

In [3]:
PHENO_PATH = "/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PHENO/all-pheno.csv"
PRS_PATH = "/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PRS/bmi.tsv.gz"
all_pheno = pd.read_csv(PHENO_PATH)
bmi_prs = pd.read_csv(PRS_PATH, sep="\t")

In [4]:
bmi_prs.indiv = bmi_prs.indiv.str.split("_", expand=True)
bmi_prs.indiv = bmi_prs.indiv.astype(int)
bmi_prs = bmi_prs.set_index('indiv')
all_pheno = all_pheno.set_index('ID')
post_all_pheno = all_pheno.reindex(bmi_prs.index)
post_all_pheno = post_all_pheno.dropna(subset=['bmi'])
bmi_prs = bmi_prs.dropna(subset=['MEAN'])
bmi_prs = bmi_prs.loc[post_all_pheno.index, :]
all_info = pd.concat([post_all_pheno, bmi_prs], axis=1)

In [5]:
all_info

,SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,QUANTILE_49,QUANTILE_54,QUANTILE_60,QUANTILE_65,QUANTILE_70,QUANTILE_75,QUANTILE_80,QUANTILE_85,QUANTILE_90,QUANTILE_95
indiv,,,,,,,,,,,,,,,,,,,,,
4305423,1.0,41.0,-12.17250,5.39163,-1.281030,0.841765,-5.26521,-1.786570,3.109920,-2.630850,...,0.69168,0.72690,0.77692,0.85104,0.90809,1.00511,1.09551,1.17859,1.32097,1.43672
4360126,0.0,46.0,-13.02450,6.41514,-0.183365,2.927610,-5.88964,0.940534,1.141060,-1.982130,...,0.98442,1.07051,1.11293,1.16987,1.23073,1.31662,1.38882,1.49347,1.62518,1.86918
4187429,0.0,52.0,-11.47120,3.48383,-1.154580,3.083830,7.65160,-0.913399,-1.548790,1.478930,...,-0.03267,0.01104,0.08616,0.15838,0.22576,0.31532,0.41725,0.49668,0.62957,0.75810
1004389,0.0,65.0,-12.13270,4.02976,-0.988080,0.750294,-2.36431,0.431658,-0.534071,-0.654367,...,-0.25169,-0.16845,-0.09728,-0.04126,0.03513,0.09505,0.15594,0.27193,0.43515,0.61574
1387182,0.0,56.0,-12.21710,3.50821,-1.625990,-1.226800,-5.34580,3.816790,0.579155,-1.169580,...,0.37442,0.42963,0.48090,0.54219,0.62524,0.68775,0.80267,0.86993,0.98681,1.11161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498323,0.0,56.0,-10.13780,3.40379,1.465470,-4.815010,-3.59211,1.318030,-0.778630,-1.225560,...,0.18317,0.23521,0.27820,0.33164,0.39136,0.45891,0.53437,0.64598,0.78231,0.94248
4386997,0.0,62.0,-9.21323,2.04166,-2.013420,0.472985,-1.67715,1.026240,0.438598,0.328357,...,0.72838,0.79457,0.85591,0.93544,0.98634,1.07220,1.13859,1.20815,1.28696,1.49401
4647541,1.0,60.0,-9.33563,5.06051,0.665526,-0.999905,-5.45859,-0.750495,3.288820,0.131781,...,0.48621,0.54190,0.58951,0.66837,0.70734,0.78434,0.86506,0.98754,1.08589,1.26673


In [6]:
res1 = admix_prs.stratify_calculate_r2(all_info, 'bmi', 'MEAN', 'SEX')
res2 = admix_prs.stratify_calculate_r2(all_info, 'bmi', 'MEAN', 'AGE')

In [7]:
res1

,level,R2,R2_std
0,1,0.240914,0.001453
1,2,0.222159,0.001813


In [8]:
res2

,level,R2,R2_std
0,1,0.233664,0.002153
1,2,0.239043,0.001527
2,3,0.237312,0.002257
3,4,0.229375,0.002085
4,5,0.221242,0.002149
